<a href="https://colab.research.google.com/github/rishabhd786/Pointnet_Pytorch/blob/master/pointnet_modelnet10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math
import random
import os
import torch
import torch.nn as nn
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import plotly.graph_objects as go
import plotly.express as px

In [2]:
!pip install path.py;
from path import Path
%cd drive/My\ Drive

/content/drive/My Drive


In [0]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip

--2020-05-24 06:55:24--  http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Resolving 3dvision.princeton.edu (3dvision.princeton.edu)... 128.112.136.61
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473402300 (451M) [application/zip]
Saving to: ‘ModelNet10.zip’

ModelNet10.zip      100%[===================>] 451.47M  36.0MB/s    in 14s     

2020-05-24 06:55:38 (33.2 MB/s) - ‘ModelNet10.zip’ saved [473402300/473402300]



In [0]:
!unzip -q ModelNet10.zip;

In [0]:
path=Path("ModelNet10")

In [4]:
folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
print(classes)

{'bathtub': 0, 'bed': 1, 'chair': 2, 'desk': 3, 'dresser': 4, 'monitor': 5, 'night_stand': 6, 'sofa': 7, 'table': 8, 'toilet': 9}


In [0]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces

In [0]:
with open(path/"bathtub/train/bathtub_0001.off", 'r') as f:
  verts1, faces1 = read_off(f)
i,j,k = np.array(faces1).T
x,y,z = np.array(verts1).T

In [0]:
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig

In [0]:
visualize_rotate([go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50, i=i,j=j,k=k)]).show()

In [0]:
def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=1,
                      line=dict(width=1,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()

#pcshow(x,y,z)   

In [0]:
def area(p1,p2,p3):
   side_a = np.linalg.norm(p1 - p2)
   side_b = np.linalg.norm(p2 - p3)
   side_c = np.linalg.norm(p3 - p1)
   s = 0.5 * ( side_a + side_b + side_c)
   return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

In [0]:
def face_sample(verts,faces,n_faces):
  areas = np.zeros((len(faces)))
  verts = np.array(verts)
  for i in range(len(faces)):
    areas[i]=area(verts[faces[i][0]],verts[faces[i][1]],verts[faces[i][2]])

  sampled_faces = (random.choices(faces, weights=areas,cum_weights=None,k=n_faces))
  return sampled_faces

In [0]:
def point_sample(data,n_points):
  verts, faces = read_off(data)
  sampled_faces=face_sample(verts,faces,n_points)
  sampled_faces=np.array(sampled_faces)
  verts = np.array(verts)
  sampled_points=np.zeros((n_points,3))
  for i in range(n_points):
    s, t = sorted([random.random(), random.random()])
    f = lambda j: s *verts[sampled_faces[i][0]][j]  + (t-s)*verts[sampled_faces[i][1]][j] + (1-t)*verts[sampled_faces[i][2]][j]
    sampled_points[i][0]=f(0)
    sampled_points[i][1]=f(1)
    sampled_points[i][2]=f(2)

  return sampled_points


  


In [0]:
with open("ModelNet10/bathtub/train/bathtub_0001.off", 'r') as f:
  sample=point_sample(f,1024)

x,y,z=np.array(sample).T

In [0]:
pcshow(x,y,z)

In [0]:
def normalise(sampled_points):
  x,y,z=np.array(sampled_points).T
  x=x-np.mean(x)
  y=y-np.mean(y)
  z=z-np.mean(z)
  x1=x/np.max((x**2+y**2+z**2)**0.5)
  y1=y/np.max((x**2+y**2+z**2)**0.5)
  z1=z/np.max((x**2+y**2+z**2)**0.5)

  return np.array([x1,y1,z1])

In [0]:
x,y,z=normalise(sample)
pcshow(x,y,z)

In [0]:
def rotate(normalised_points):
   x,y,z=normalised_points
   theta = random.random() * 2. * math.pi
   rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),0],[ math.sin(theta),math.cos(theta),0],[0, 0,1]])
   arr= np.array([x,y,z])
   rot_pointcloud = rot_matrix.dot(arr)
  # print(np.shape(rot_pointcloud[0]))
   return  rot_pointcloud

In [0]:
x,y,z=rotate(x,y,z)

TypeError: ignored

In [0]:
pcshow(x,y,z)

In [0]:
def add_noise(rot_cloud):
  x,y,z=rot_cloud
  noisex = np.random.normal(0, 0.02, (1024,))
  noisey = np.random.normal(0, 0.02, (1024,))
  noisez = np.random.normal(0, 0.02, (1024,))  
  x=x+noisex
  y=y+noisey
  z=z+noisez
  return np.array([x,y,z])

In [0]:
x,y,z=add_noise(x,y,z)
pcshow(x,y,z)

In [0]:
def data_creator(root,n_points,folder):
  folders = [dir for dir in sorted(os.listdir(root)) if os.path.isdir(root/dir)]
  classes = {folder: i for i, folder in enumerate(folders)}
  files = []
  cloud=[]
  for category in classes.keys():
            new_dir = root/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    sample['pcd_path'] = new_dir/file
                    sample['category'] = category
                    files.append(sample)
  
  for i,_ in enumerate(files):
    pcd_path = files[i]['pcd_path']
    category = files[i]['category']
    with open(pcd_path, 'r') as f:
     out=point_sample(f,n_points)
    out=normalise(out)
    out=rotate(out)
    out=add_noise(out)
    samp={}
    samp["pointcloud"]=out
    samp['category']=category
    cloud.append(samp)
    print(i)
    
  return cloud  
  


In [0]:
datatr=data_creator(path,1024,"train")

In [0]:
datatr=np.array(datatr)
#x,y,z=data[0]['pointcloud']

In [21]:
inv_classes = {i: cat for cat, i in classes.items()};
inv_classes

{0: 'bathtub',
 1: 'bed',
 2: 'chair',
 3: 'desk',
 4: 'dresser',
 5: 'monitor',
 6: 'night_stand',
 7: 'sofa',
 8: 'table',
 9: 'toilet'}

In [0]:
ds=[]
for i in range(len(datatr)):
  x,y,z=datatr[i]['pointcloud']
  label=datatr[i]['category']
  label=classes[label]
  arr=np.array([x,y,z])
  arr=torch.from_numpy(arr)
  sam={}
  sam['pc']=arr
  sam['lab']=label
  ds.append(sam)




train_loader = DataLoader(dataset=ds, batch_size=32, shuffle=True)

In [0]:
data_test=data_creator(path,1024,"test")
data_test=np.array(data_test)

In [0]:
ds_test=[]
for i in range(len(data_test)):
  x,y,z=data_test[i]['pointcloud']
  label=data_test[i]['category']
  label=classes[label]
  arr=np.array([x,y,z])
  arr=torch.from_numpy(arr)
  sam={}
  sam['pc']=arr
  sam['lab']=label
  ds_test.append(sam)




test_loader = DataLoader(dataset=ds_test, batch_size=32,shuffle=False)

In [24]:
real_batch = next(iter(test_loader))
labe=real_batch['lab']
print(labe.size(0))

32


In [0]:
class STransform(nn.Module):
  def __init__(self,n_points):
    super().__init__()
    self.conv1=nn.Conv1d(3,64,1)
    self.bn1=nn.BatchNorm1d(64)
    self.relu=nn.ReLU()
    self.conv2=nn.Conv1d(64,128,1)
    self.bn2=nn.BatchNorm1d(128)
    self.conv3=nn.Conv1d(128,1024,1)
    self.bn3=nn.BatchNorm1d(1024)
    self.maxpool=nn.MaxPool1d(1024)
    self.fc1=nn.Linear(1024,512)
    self.bn4=nn.BatchNorm1d(512)
    self.fc2=nn.Linear(512,256)
    self.bn5=nn.BatchNorm1d(256)
    self.fc3=nn.Linear(256,9)
    
  def forward(self,x):
    #b*3*1024
    bs=x.size(0)
    out=self.relu(self.bn1(self.conv1(x)))
    out=self.relu(self.bn2(self.conv2(out)))
    out=self.relu(self.bn3(self.conv3(out)))
    out=self.maxpool(out)
    #b*1024*1
    out=out.view(-1,1024)
    out=self.relu(self.bn4(self.fc1(out)))
    out=self.relu(self.bn5(self.fc2(out)))
    out=self.fc3(out)
    out=out.view(-1,3,3)
    init = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    out=out+init.cuda()

    return out

In [0]:
class FTransform(nn.Module):
  def __init__(self,n_points):
    super().__init__()
    self.conv1=nn.Conv1d(64,64,1)
    self.bn1=nn.BatchNorm1d(64)
    self.relu=nn.ReLU()
    self.conv2=nn.Conv1d(64,128,1)
    self.bn2=nn.BatchNorm1d(128)
    self.conv3=nn.Conv1d(128,1024,1)
    self.bn3=nn.BatchNorm1d(1024)
    self.maxpool=nn.MaxPool1d(1024)
    self.fc1=nn.Linear(1024,512)
    self.bn4=nn.BatchNorm1d(512)
    self.fc2=nn.Linear(512,256)
    self.bn5=nn.BatchNorm1d(256)
    self.fc3=nn.Linear(256,4096)

  def forward(self,x):
    #b*3*1024
    bs=x.size(0)
    out=self.relu(self.bn1(self.conv1(x)))
    out=self.relu(self.bn2(self.conv2(out)))
    out=self.relu(self.bn3(self.conv3(out)))
    out=self.maxpool(out)
    #b*1024*1
    out=out.view(-1,1024)
    out=self.relu(self.bn4(self.fc1(out)))
    out=self.relu(self.bn5(self.fc2(out)))
    out=self.fc3(out)
    out=out.view(-1,64,64)
    init = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    out=out+init.cuda()

    return out

In [0]:
class Net(nn.Module):
  def __init__(self,n_points):
    super().__init__()
    self.n_points=n_points
    self.trans3=STransform(self.n_points).to(device)
    self.trans64=FTransform(self.n_points).to (device)
    self.conv1=nn.Conv1d(3,64,1)
    self.bn1=nn.BatchNorm1d(64)
    self.relu=nn.ReLU()
    self.conv2=nn.Conv1d(64,64,1)
    self.bn2=nn.BatchNorm1d(64)
    self.conv3=nn.Conv1d(64,64,1)
    self.bn3=nn.BatchNorm1d(64)
    self.conv4=nn.Conv1d(64,128,1)
    self.bn4=nn.BatchNorm1d(128)
    self.conv5=nn.Conv1d(128,1024,1)
    self.bn5=nn.BatchNorm1d(1024)
    self.maxpool=nn.MaxPool1d(1024)
    self.fc1=nn.Linear(1024,512)
    self.bn6=nn.BatchNorm1d(512)
    self.fc2=nn.Linear(512,256)
    self.bn7=nn.BatchNorm1d(256)
    self.drop=nn.Dropout(0.3)
    self.fc3=nn.Linear(256,10)
    self.softmax=nn.Softmax(1)

  def forward(self,x):
    ST=self.trans3(x)
    out=torch.bmm(torch.transpose(x,1,2),ST)
    out=out.view(-1,3,1024)
    out=self.relu(self.bn1(self.conv1(out)))
    out=self.relu(self.bn2(self.conv2(out)))
    FT=self.trans64(out)
    out=torch.bmm(torch.transpose(out,1,2),FT)
    out=out.view(-1,64,1024)
    out=self.relu(self.bn3(self.conv3(out)))
    out=self.relu(self.bn4(self.conv4(out)))
    out=self.relu(self.bn5(self.conv5(out)))
    out=self.maxpool(out)
    out=out.view(-1,1024)
    out=self.relu(self.bn6(self.fc1(out)))
    out=self.relu(self.bn7(self.drop(self.fc2(out))))
    out=self.fc3(out)

    return out,FT
    





In [0]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [46]:
summary(model,(3,1024),batch_size=32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [32, 64, 1024]             256
       BatchNorm1d-2             [32, 64, 1024]             128
              ReLU-3             [32, 64, 1024]               0
            Conv1d-4            [32, 128, 1024]           8,320
       BatchNorm1d-5            [32, 128, 1024]             256
              ReLU-6            [32, 128, 1024]               0
            Conv1d-7           [32, 1024, 1024]         132,096
       BatchNorm1d-8           [32, 1024, 1024]           2,048
              ReLU-9           [32, 1024, 1024]               0
        MaxPool1d-10              [32, 1024, 1]               0
           Linear-11                  [32, 512]         524,800
      BatchNorm1d-12                  [32, 512]           1,024
             ReLU-13                  [32, 512]               0
           Linear-14                  [

In [0]:
def loss(out,label,mat64,bs):
  criterion=nn.CrossEntropyLoss().to(device)
  id=torch.eye(64, requires_grad=True).repeat(bs,1,1).to(device)
  reg=id-torch.bmm(mat64,torch.transpose(mat64,1,2)).to(device)
  los=criterion(out,label)+0.001*torch.norm(reg)/float(bs)
  #los=criterion(out,label)
  return los

In [0]:
epochs=100
lr=0.001
model=Net(1024).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [47]:
total_step = len(train_loader)
for epoch in range(epochs):
    for i, data in enumerate(train_loader):
        points,label=data['pc'].to(device).float(),data['lab'].to(device)
        optimizer.zero_grad()
        out,mat64=model(points)
        
        bs=out.size(0)
        losse=loss(out,label,mat64,bs)
        
        losse.backward()
        optimizer.step()
        if (i+1) % 25 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1,epochs, i+1, total_step, losse.item()))  
    if (epoch+1)%20==0:
        lr=lr/2.0
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for datate in test_loader:
          points,label=datate['pc'].to(device).float(),datate['lab'].to(device)
          outputs,_ = model(points)
          _, predicted = torch.max(outputs.data, 1)
          total += label.size(0)
          correct += (predicted == label).sum().item()

        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))   
       

Epoch [1/100], Step [25/125] Loss: 0.0352
Epoch [1/100], Step [50/125] Loss: 0.1999
Epoch [1/100], Step [75/125] Loss: 0.0050
Epoch [1/100], Step [100/125] Loss: 0.0707
Epoch [1/100], Step [125/125] Loss: 0.0079
Accuracy of the model on the test images: 64.09691629955947 %
Epoch [2/100], Step [25/125] Loss: 0.0047
Epoch [2/100], Step [50/125] Loss: 0.0846
Epoch [2/100], Step [75/125] Loss: 0.1666
Epoch [2/100], Step [100/125] Loss: 0.0038
Epoch [2/100], Step [125/125] Loss: 0.0156
Accuracy of the model on the test images: 61.23348017621145 %
Epoch [3/100], Step [25/125] Loss: 0.0306
Epoch [3/100], Step [50/125] Loss: 0.0387
Epoch [3/100], Step [75/125] Loss: 0.0632
Epoch [3/100], Step [100/125] Loss: 0.0207
Epoch [3/100], Step [125/125] Loss: 0.0105
Accuracy of the model on the test images: 59.030837004405285 %
Epoch [4/100], Step [25/125] Loss: 0.3694
Epoch [4/100], Step [50/125] Loss: 0.1019
Epoch [4/100], Step [75/125] Loss: 0.0926
Epoch [4/100], Step [100/125] Loss: 0.0996
Epoch [4

KeyboardInterrupt: ignored